1. Załaduj dane \\
Wczytaj plik people_wiki.csv. Składa się on z trzech kolumn URI, imienia i nazwiska osoby, oraz tekstu jej biografii z wikipedii (po angielsku).

In [1]:
import pandas as pd

file_path = "people_wiki.csv"

#file_path = "https://github.com/PawelCedzich/NLPLabs/blob/main/Lista3/people_wiki.csv"

def load_people_wiki(file_path):
    try:
        df = pd.read_csv(file_path, sep=",", encoding="utf-8", on_bad_lines="skip", nrows=1000)
        print("Plik został wczytany poprawnie.")
        print(df.head())
        return df
    except Exception as e:
        print(f"Błąd podczas wczytywania pliku: {e}")
        return None

data = load_people_wiki(file_path)


Plik został wczytany poprawnie.
                                                 URI                 name  \
0        <http://dbpedia.org/resource/Digby_Morrell>        Digby Morrell   
1       <http://dbpedia.org/resource/Alfred_J._Lewy>       Alfred J. Lewy   
2        <http://dbpedia.org/resource/Harpdog_Brown>        Harpdog Brown   
3  <http://dbpedia.org/resource/Franz_Rottensteiner>  Franz Rottensteiner   
4               <http://dbpedia.org/resource/G-Enka>               G-Enka   

                                                text  
0  digby morrell born 10 october 1979 is a former...  
1  alfred j lewy aka sandy lewy graduated from un...  
2  harpdog brown is a singer and harmonica player...  
3  franz rottensteiner born in waidmannsfeld lowe...  
4  henry krvits born 30 december 1974 in tallinn ...  


In [2]:
data.head(10)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...
6,<http://dbpedia.org/resource/Aaron_LaCrate>,Aaron LaCrate,aaron lacrate is an american music producer re...
7,<http://dbpedia.org/resource/Trevor_Ferguson>,Trevor Ferguson,trevor ferguson aka john farrow born 11 novemb...
8,<http://dbpedia.org/resource/Grant_Nelson>,Grant Nelson,grant nelson born 27 april 1971 in london also...
9,<http://dbpedia.org/resource/Cathy_Caruth>,Cathy Caruth,cathy caruth born 1955 is frank h t rhodes pro...


2. Przetwórz dane \\
Tokenizacja: Podziel tekst na poszczególne słowa. \\
Zamień wszystkie słowa na małe litery. \\
Usuń słowa, które nie niosą znaczącej informacji (np. "and", "or"). \\
Stemming lub lematyzacja: Zredukuj słowa do ich podstawowej formy.

Przetwórz zapytanie: "graduated at Harvard"

In [17]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
def preprocess_text(text):
    # Tokenizacja: Podziel tekst na poszczególne słowa
    tokens = word_tokenize(text)

    # Zamień wszystkie słowa na małe litery
    tokens = [word.lower() for word in tokens]

    # Usuń słowa, które nie niosą znaczącej informacji (np. "and", "or")
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming: Zredukuj słowa do ich podstawowej formy
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    return tokens

def preprocess_dataframe(df):
    # Przetwórz tekst w kolumnie biografii
    df['processed_biography'] = df['text'].apply(preprocess_text)
    return df

if data is not None:
    # Preprocess the data
    processed_data = preprocess_dataframe(data)

    # Print the processed data
    print(processed_data.head())

                                                 URI                 name  \
0        <http://dbpedia.org/resource/Digby_Morrell>        Digby Morrell   
1       <http://dbpedia.org/resource/Alfred_J._Lewy>       Alfred J. Lewy   
2        <http://dbpedia.org/resource/Harpdog_Brown>        Harpdog Brown   
3  <http://dbpedia.org/resource/Franz_Rottensteiner>  Franz Rottensteiner   
4               <http://dbpedia.org/resource/G-Enka>               G-Enka   

                                                text  \
0  digby morrell born 10 october 1979 is a former...   
1  alfred j lewy aka sandy lewy graduated from un...   
2  harpdog brown is a singer and harmonica player...   
3  franz rottensteiner born in waidmannsfeld lowe...   
4  henry krvits born 30 december 1974 in tallinn ...   

                                      processed_text  \
0  [digbi, morrel, born, 10, octob, 1979, former,...   
1  [alfr, j, lewi, aka, sandi, lewi, graduat, uni...   
2  [harpdog, brown, singer, harm

3. Oblicz TFIDF \\
Wykorzystując TFIDF oblicz wagi oraz zwektoryzuj je

In [15]:
biographies = data["text"].tolist()

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(biographies)

# Konwersja macierzy TFIDF do DataFrame dla lepszej czytelności
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Wyświetlenie wag TFIDF
print("TFIDF Weights:")
print(tfidf_df)

# Przetworzenie zapytania "graduated at Harvard"
query = "graduated at Harvard"
query_tfidf = vectorizer.transform([query])

# Konwersja wektora TFIDF zapytania do DataFrame dla lepszej czytelności
query_tfidf_df = pd.DataFrame(query_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Wyświetlenie wag TFIDF zapytania
print("\nTFIDF Weights for the query 'graduated at Harvard':")
print(query_tfidf_df)

TFIDF Weights:
      00  000  000577  005  009   01   02  020  0200  02182748  ...  zurab  \
0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
1    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
2    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
3    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
4    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
..   ...  ...     ...  ...  ...  ...  ...  ...   ...       ...  ...    ...   
995  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
996  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
997  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
998  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   
999  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0       0.0  ...    0.0   

     zurich  zuzana  zviad  zvonimir  zwarts  zw

4. Dla podanego zapytania znajdź 10 najbliższych mu osób obliczając podobieństwo cosinusów albo euklidesowe.

In [19]:
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

# Pobranie indeksów 10 najbardziej podobnych biografii
top_10_indices = cosine_similarities.argsort()[-10:][::-1]

# Wyświetlenie 10 najbardziej podobnych biografii
print("Top 10 most similar biographies to the query 'graduated at Harvard':")
for idx in top_10_indices:
    print(f"URI: {data['URI'][idx]}")
    print(f"name: {data['name'][idx]}")
    print(f"Biography: {data['text'][idx]}")
    print(f"Cosine Similarity: {cosine_similarities[idx]}")
    print()

Top 10 most similar biographies to the query 'graduated at Harvard':
URI: <http://dbpedia.org/resource/Ronald_S._Sullivan_Jr.>
name: Ronald S. Sullivan Jr.
Biography: ronald s sullivan jr born december 12 1966 in gary indiana is a law professor at the harvard law school sullivan graduated phi beta kappa from morehouse college in 1989 and received his jd from harvard law school in 1994 prior to joining the harvard law school faculty sullivan served as the director of the public defender service for the district of columbiasullivan was elected president of the black law students association at harvard law school and served as a general editor of the harvard black letter law journal now known as the civil rights and civil liberties journal prior to joining the harvard faculty sullivan worked with the dc law firms of baach robinson lewis and skadden arps slate meagher flom where sullivan quickly gained experience with highprofile cases such as those involving president clintonsullivan bega